<a href="https://colab.research.google.com/github/SharlotteManganye/Deep-Learning-CNN/blob/main/PuNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

class ProductUnitNN(nn.Module):
    def __init__(self, input_dim, hid_dim, output_dim):
        super(ProductUnitNN, self).__init__()
        self.input_dim = input_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim

        # Initialize weights
        self.w1 = nn.Parameter(torch.randn(input_dim, hid_dim), requires_grad=True)
        self.w2 = nn.Parameter(torch.randn(hid_dim, output_dim), requires_grad=True)
        self.bias = nn.Parameter(torch.ones(output_dim) * -1, requires_grad=True)

    def forward(self, x):
        # Adjust net_sig computation
        net_sig = torch.zeros((x.size(0), self.hid_dim), device=x.device)
        for i in range(self.input_dim):
            condition = x[:, i].unsqueeze(1) != 0
            values_to_add = torch.sum(self.w1 * torch.log(torch.abs(x.unsqueeze(2) - 1)), dim=1)
            # Expand condition to match net_sig's shape for broadcasting
            condition = condition.expand_as(net_sig)  # Becomes (batch_size, hid_dim)
            net_sig = torch.where(condition, values_to_add, net_sig)
        z = net_sig @ self.w2 + self.bias
        return z

# Training settings
batch_size = 64
learning_rate = 0.001
epochs = 10

# MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Model, loss function, optimizer
input_dim = 784  # 28x28 images flattened
hid_dim = 100    # Example hidden dimension
output_dim = 10  # 10 classes for MNIST

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ProductUnitNN(input_dim, hid_dim, output_dim)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Store gradient norms
grad_norms = {name: [] for name, param in model.named_parameters()}

# Training the model
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.view(-1, input_dim).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()

        # Calculate and store gradient norms
        for name, param in model.named_parameters():
            if param.grad is not None:
                grad_norms[name].append(param.grad.norm().item())

        optimizer.step()
        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Testing the model
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.view(-1, input_dim).to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)\n')

for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

# Plot gradient norms
for name, norms in grad_norms.items():
    plt.plot(norms, label=name)
plt.xlabel('Batch number')
plt.ylabel('Gradient norm')
plt.legend()
plt.show()


Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15829507.69it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 475616.41it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3780528.77it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 8787144.27it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train Epoch: 1 [0/60000 (0%)]	Loss: 130.874069
Train Epoch: 1 [3200/60000 (5%)]	Loss: 175.440842
Train Epoch: 1 [6400/60000 (11%)]	Loss: 123.531250
Train Epoch: 1 [9600/60000 (16%)]	Loss: 80.761650
Train Epoch: 1 [12800/60000 (21%)]	Loss: 114.750122
Train Epoch: 1 [16000/60000 (27%)]	Loss: 108.515099
Train Epoch: 1 [19200/60000 (32%)]	Loss: 94.863319
Train Epoch: 1 [22400/60000 (37%)]	Loss: 64.042168
Train Epoch: 1 [25600/60000 (43%)]	Loss: 105.106049
Train Epoch: 1 [28800/60000 (48%)]	Loss: 68.801155
Train Epoch: 1 [32000/60000 (53%)]	Loss: 68.669281
Train Epoch: 1 [35200/60000 (59%)]	Loss: 29.518572
Train Epoch: 1 [38400/60000 (64%)]	Loss: 52.073051
Train Epoch: 1 [41600/60000 (69%)]	Loss: 39.826523
Train Epoch: 1 [44800/60000 (75%)]	Loss: 53.541702
Train Epoch: 1 [48000/60000 (80%)]	Loss: 23.297148
Train Epoch: 1 [51200/60000 (85%)]	Loss: 27.150322
Train Epoch: 1 [54400/60000 (91%)]	Loss: 19.769260
Train Epoc